# 1.0 Um problema de classificação de ponta-a-ponta usando NLP

## 1.1 Descrição do dataset

O dataset contém informações gerais de 5.000 processos julgados
nos Juizados Especiais Federais dentro das Seções Judiciárias do Tribunal Regional Federal da 5a Região (https://www.trf5.jus.br). Os dados são oriundos da raspagem da consulta pública processual. Além disso, ele possui 46 colunas, das quais duas possuem texto livre:
"conteudo_sentenca" e "conteudo_acordao".

O dataset pode ser baixado no link a seguir: https://jacob.al/dataset_juizados 

Ao longo dos notebooks, vão ser realizados os seguintes passos:

1. Importação do dataset **(concluído)**
2. Análise exploratória dos dados **(concluído)**
3. Pré-processamento **(concluído)**
4. Verificação dos dados **(concluído)**
5. Segregação dos dados **(concluído)**
6. Treinamento
7. Teste

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>


## 1.2 Instalação e importação das bibliotecas

In [ ]:
!pip install wandb

In [ ]:
import logging
import wandb
import pandas as pd
import numpy as np
import joblib
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer

In [ ]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.3 Configuração do holdout

In [ ]:
# variáveis globais

# proporção utilizada para dividir entre dados de teste e de treinamento
val_size = 0.30

# Semente utilizada para propósitos de reproducibilidade
seed = 41

# coluna de referência para dividir os dados
stratify = "assunto_cnj"

# nome do artefato de entrada
artifact_input_name = "nlp_bolsa/train.csv:latest"

# tipo do artefato
artifact_type = "Train"

In [ ]:
# configuração do logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# objeto do logging
logger = logging.getLogger()

# Inicialização do projeto do wandb
run = wandb.init(project="nlp_bolsa",job_type="train")

logger.info("Downloading and reading train artifact")
local_path = run.use_artifact(artifact_input_name).file()
df_train = pd.read_csv(local_path)


# Divisão do arquivo train.csv em treinamento e validação
logger.info("Spliting data into train/val")
x_train, x_val, y_train, y_val = train_test_split(df_train.drop(labels=stratify,axis=1),
                                                  df_train[stratify],
                                                  test_size=val_size,
                                                  random_state=seed,
                                                  shuffle=True,
                                                  stratify=df_train[stratify])

17-07-2022 18:50:57 Downloading and reading train artifact
17-07-2022 18:50:58 Spliting data into train/val


In [ ]:
logger.info("x train: {}".format(x_train.shape))
logger.info("y train: {}".format(y_train.shape))
logger.info("x val: {}".format(x_val.shape))
logger.info("y val: {}".format(y_val.shape))

17-07-2022 18:50:59 x train: (2449, 1)
17-07-2022 18:50:59 y train: (2449,)
17-07-2022 18:50:59 x val: (1050, 1)
17-07-2022 18:50:59 y val: (1050,)


## 1.4 Prepapração dos dados

### 1.4.1 Encoding a variável alvo

Usada na parte da modelagem com redes neurais

In [ ]:
y_train.head(10)

1171    Direito Previdenciário
1821    Direito Previdenciário
863     Direito Previdenciário
1135    Direito Previdenciário
1126    Direito Previdenciário
1023    Direito Previdenciário
2166    Direito Previdenciário
628     Direito Previdenciário
2169    Direito Previdenciário
214     Direito Previdenciário
Name: assunto_cnj, dtype: object

In [ ]:
ohe = OneHotEncoder()

y_train = ohe.fit_transform(y_train.values.reshape(-1,1))
y_train

<2449x7 sparse matrix of type '<class 'numpy.float64'>'
	with 2449 stored elements in Compressed Sparse Row format>

In [ ]:
matrix= np.zeros((7,7))
for i in range(7):
  for j in range(7):
    if i == j:
      matrix[i,i] = 1

inverse = ohe.inverse_transform(matrix)

In [ ]:
print(matrix , '=', inverse)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]] = [['Direito Administrativo e outras matérias do Direito Público']
 ['Direito Civil']
 ['Direito Previdenciário']
 ['Direito Processual Civil e do Trabalho']
 ['Direito Tributário']
 ['Direito do Consumidor']
 ['Direito do Trabalho']]


### 1.4.2 Pipeline completo

#### 1.4.2.1 Extrator de features

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # construtor da classe
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.feature_names]

#### 1.4.2.2 Tratamento de features categóricas

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def doc_to_list(text_df, list_words):
  for i in range(text_df.shape[0]):
    list_words.append(text_df.loc[i, 'conteudo_sentenca'])

  return list_words

In [ ]:
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [ ]:
# Tratando variáveis categóricas
class CategoricalTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames

    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self):
        return self.colnames.tolist()

    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)

        # Removendo a pontuação
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: re.sub('-', ' ', x))
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: re.sub('[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', x))

        # Colocando todos os textos em minúsuculo
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: x.lower())

        # Removendo caracteres especiais e stopwords
        stop = stopwords.words('portuguese')

        # palavras comuns em processos jurídicos
        stop.extend(['artigo', 'lei', 'sentença', 'município', 'nacional', '1o', 'art', 
             'autora', 'parte', 'honorários', 'advocatícios', 'termo', 'tempo',
             'justiça', 'etc', 'n°', '°', 'termos', 'parágrafo'])
        pat = r'\b(?:{}/-;)\b'.format('|'.join(stop))
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(pat, '')

        # Removendo dígitos
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(r'\d+', ' ')

        # Removendo sub-string
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(r'\s+', ' ')

        # Pegando os tokens das palavras
        df['conteudo_sentenca'].apply(word_tokenize)
        
        ''' Usada na parte da modelagem com redes neurais
        # Pegando os tokens das palavras
        list_words = []
        list_words = doc_to_list(df, list_words)

        # criando o tokenizer
        tokenizer = create_tokenizer(list_words)

        df = tokenizer.texts_to_matrix(df.loc[:,'conteudo_sentenca'], mode='freq')
        '''
        return df


In [ ]:
fs = FeatureSelector(x_train.select_dtypes("object").columns.to_list())
df = fs.fit_transform(x_train)
df.head()

,conteudo_sentenca
1171,"SENTENÇA I. RELATÓRIO. Dispensado o relatório,..."
1821,TERMO DE ACORDO PROCESSO: 0502544-12.2018.4.0...
863,SENTENÇA Trata-se de ação especial cível afora...
1135,SENTENÇA RELATÓRIO Trata-se de ação especial ...
1126,SENTENÇA - TIPO A Dispensado o relatório (art....


In [ ]:
ct = CategoricalTransformer(new_features=True,colnames=df.columns.tolist())
df_cat = ct.fit_transform(df)
df_cat

,conteudo_sentenca
1171,i relatório dispensado relatório n° aplicado ...
1821,acordo processo autor antônia bento silva réu...
863,trata ação especial cível aforada contra inst...
1135,relatório trata ação especial cível promovida...
1126,tipo dispensado relatório cc passo fundamenta...
...,...
3390,audiência instrução julgamento ação especial ...
2328,i – relatório dispensado n° aplicado caso for...
1379,vistos i – relatório aparte ajuizou ação rito...
2145,tipo força disposto combinado dispenso feitur...


In [ ]:
df_cat.shape

(2449, 1)

In [ ]:
df_cat.loc[1171,'conteudo_sentenca']

' i relatório dispensado relatório n° aplicado caso força ° n° decido ii fundamentação analisando presente processo observa requisitos petição inicial constantes arts código processo civil integralmente cumpridos demandante embora sido devidamente intimada emendar petição inicial caput código processo civil assim considerando cumpriu determinação judicial dando dessa forma regular prosseguimento feito impõe extinção processo resolução mérito parágrafo único cc inc i código processo civil ressalte oportuno parágrafo primeiro aplicável estabelece extinção processo qualquer hipótese condicionada prévia intimação pessoal partes iii dispositivo exposto tudo autos consta extingo processo resolução mérito parágrafo único cc inciso i ambos código processo civil observado disposto § custas força disposto cc arts publique registre intimem após certifique trânsito julgado remetam autos arquivo baixa distribuição observadas disposições o sobral data supra iaci rolim sousa juíza federal a varasjce 

#### 1.4.2.3 Preparação dos dados com o Pipeline

In [ ]:
# Pegando as featues categóricas
categorical_features = x_train.select_dtypes("object").columns.to_list()

# Definição dos passos do pipeline categórico
categorical_pipeline = Pipeline(steps=[('cat_selector', FeatureSelector(categorical_features)),
                                       ('cat_transformer', CategoricalTransformer(colnames=categorical_features))
                                       ]
                                )

full_pipeline_preprocessing = FeatureUnion(transformer_list=[('cat_pipeline', categorical_pipeline)
                                                             ]
                              )

In [ ]:
new_data = full_pipeline_preprocessing.fit_transform(x_train)
df_new_train = pd.DataFrame(new_data, columns=['conteudo_sentenca'])
df_new_train.shape

(2449, 1)

In [ ]:
df_new_train.columns

Index(['conteudo_sentenca'], dtype='object')

In [ ]:
new_data_test = full_pipeline_preprocessing.transform(x_val)
df_new_val = pd.DataFrame(new_data_test, columns=['conteudo_sentenca'])
df_new_val.shape

(1050, 1)

In [ ]:
df_new_val.head()

,conteudo_sentenca
0,relatório dispensado relatório parágrafo únic...
1,i – relatório dispensado relatório passo deci...
2,relatório trata ação especial proposta avany ...
3,i – relatório força disposto caput combinado ...
4,i relatório dispensado n° aplicado caso força...


#### 1.4.2.4 Compilando o modelo

Usada na modelagem com redes reurais

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import SGD
from wandb.keras import WandbCallback
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import pytz

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_begin(self, batch, logs=None):
    self.begins = time.time()
    print('Training: begins at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))

  def on_train_end(self, logs=None):
    print('Training: ends at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))
    print('Duration: {:.2f} seconds'.format(time.time() - self.begins))  

In [ ]:
df_new_train.shape[1]

39667

In [ ]:
# definição do modelo
def define_model(n_words):
  # definição da rede
  model = Sequential()
  model.add(Dense(100, input_shape=(n_words,), activation='relu'))
  model.add(Dense(7, activation='softmax'))
  # compilação do modelo
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarização
  model.summary()
  
  return model

n_words = df_new_train.shape[1]
model = define_model(n_words)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               3966800   
                                                                 
 dense_1 (Dense)             (None, 7)                 707       
                                                                 
Total params: 3,967,507
Trainable params: 3,967,507
Non-trainable params: 0
_________________________________________________________________


## 1.5 Holdout Training

### 1.5.1 Keras

In [ ]:
#from scikeras.wrappers import KerasClassifier

In [ ]:
'''
# Pipeline completo
pipe = Pipeline(steps = [('full_pipeline', categorical_pipeline),
                         ("classifier",KerasClassifier(model=model, epochs=50,
                                                        callbacks=[MyCustomCallback,
                                                                   WandbCallback(log_weights=True,
                                                                                  log_gradients=True,
                                                                                  training_data=(x_train, y_train),
                                                                                  log_evaluation=True)],
                                                                                  batch_size=64, verbose=1,
                                                                                  validation_split=0.3
                                                        ))
                         ]
                )

# training
logger.info("Training")
pipe.fit(x_train, y_train.toarray())
'''

### 1.5.2 Bertopic

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

In [ ]:
topic_model = BERTopic(language="portuguese", calculate_probabilities=True, verbose=True)

In [ ]:
topics, probs = topic_model.fit_transform(df_new_train['conteudo_sentenca'])

Batches:   0%|          | 0/77 [00:00<?, ?it/s]

2022-07-17 20:27:02,835 - BERTopic - Transformed documents to Embeddings
2022-07-17 20:27:13,546 - BERTopic - Reduced dimensionality
2022-07-17 20:27:14,270 - BERTopic - Clustered reduced embeddings


In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,566,-1_benefício_especial_social_deficiência
1,0,90,0_maternidade_segurada_salário_parto
2,1,79,1_federal_juizado_judiciário_poder
3,2,73,2_processual_unidade_desenvolvimento_regular
4,3,69,3_audiência_rural_inss_mm


In [ ]:
topic_model.get_topic(0)

[('maternidade', 0.03584017552313066),
 ('segurada', 0.027353517864285855),
 ('salário', 0.026887803787582064),
 ('parto', 0.021642111703227697),
 ('rural', 0.020412054677293515),
 ('nascimento', 0.01682863323744589),
 ('prova', 0.014798725906889146),
 ('período', 0.013491935258463164),
 ('especial', 0.012275060970622599),
 ('início', 0.011545870243964279)]

In [ ]:
# Visualize topics
topic_model.visualize_topics();

In [ ]:
topic_model.visualize_distribution(probs[120], min_probability=0.005)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

A partir dos gráficos acima, é possível observar que há bastante relação entre as palavras doença, auxílio e invalizer, por exemplo. As quais provavelmente estão relacionadas aos processos cujo assunto é Direito Previdenciário, muito visto na análise exploratória dos dados. Além desses, é possível observar várias outras relações estão relacionadas com seus respectivos assuntos processuais.

# 1.6 Verificando na base de validação

In [ ]:
topics_val, probs_val = topic_model.transform(df_new_val['conteudo_sentenca'])

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

2022-07-17 20:36:23,594 - BERTopic - Reduced dimensionality
2022-07-17 20:36:23,973 - BERTopic - Calculated probabilities with HDBSCAN
2022-07-17 20:36:23,975 - BERTopic - Predicted clusters


In [ ]:
topic_model.visualize_distribution(probs_val[150], min_probability=0.005)

In [ ]:
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_encoder = "target_encoder"
artifact_model = "model_export"

In [ ]:
logger.info("Dumping the artifacts to disk")
# Save the model using joblib
joblib.dump(topic_model, artifact_model)

17-07-2022 20:48:59 Dumping the artifacts to disk


['model_export']

In [ ]:
# Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="A topic model for NLP"
                          )

logger.info("Logging model artifact")
artifact.add_file(artifact_model)
run.log_artifact(artifact)

17-07-2022 20:49:04 Logging model artifact


In [ ]:
# close the current run before to execute the next section
run.finish()